In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [66]:
my $practice_input1 = qq/
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
/;
my $practice_input2 = qq/
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
/;


#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################


In [46]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub mulp { my ($a, $v) = @_; return [ map { $a->[$_] * $v } 0 .. $#$a ]; }
sub stringp { my $p = $_[0] // $_; return join ',', @$p }
sub rotate90p { my $p = $_[0] // $_; return [$p->[1], -$p->[0]] }
sub rotate270p { my $p = $_[0] // $_; return [-$p->[1], $p->[0]] }
sub uniqp { my %h; @h{map { join ',', @$_ } @_} = (); map [ split ',' ], keys %h }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    my @adj;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}

Warning: Subroutine subp redefined at reply input line 1.

Subroutine addp redefined at reply input line 2.

Subroutine mulp redefined at reply input line 3.

Subroutine stringp redefined at reply input line 4.

Subroutine uniqp redefined at reply input line 7.

Subroutine in_bounds redefined at reply input line 8.

Subroutine adjacent_points redefined at reply input line 12.


In [ ]:
$graphql::graphql_methods{flood_group} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};
        my @shape = shape($self->{arr});

        state %flood_group_cache;

        return $flood_group_cache{$arr}{stringp($self->{coord})} if exists $flood_group_cache{$arr}{stringp($self->{coord})};
        
        my $key = $self->{value};
        my %group;
        my @next_points = ($self->{coord});
        my $rep = 0;
        while (@next_points) {
            die if $rep++ > 50;
            @group{map stringp, @next_points} = ();
            @next_points =
                uniqp
                grep { not exists $group{stringp $_} }
                grep $arr->[$_->[0]][$_->[1]] eq $key,
                map adjacent_points($_, @shape),
                @next_points;
        }
        my $ret = [ map [split ','], keys %group ];
        foreach (@$ret) {
            $flood_group_cache{$arr}{stringp $_} = $ret;
        }
        return $ret;
    };

In [167]:
sub string_2d_numbers_array {
    my ($arr) = @_;
    return join "\n", map { join '', map { defined ? sprintf "% 6d|", $_ : '      |' } @$_ } @$arr;
}

$graphql::graphql_methods{map_distance_reindeer} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};

        # cache to prevent fetching the same data many times
        state %map_distance_cache;
        return $map_distance_cache{$arr}{stringp($self->{coord})} if exists $map_distance_cache{$arr};

        # prep map data
        my @shape = shape($self->{arr});
        my @start =
            grep defined, flatten_nd
            map_nd selector('where.coord'),
            map_nd_indexed graphql_query(qq/{ where(value:"$params->{from_value}") { coord } }/),
            $arr;

        # iterator state data
        my %group;
        my %cost_map;
        my @next_points = map [ $_, [0,1], 0 ], @start;
        my $steps = 0;
        while (@next_points) {
            last if $steps++ > 100000;
            @next_points = sort { $a->[2] <=> $b->[2] } @next_points;
            my $state = shift @next_points;
            my ($p, $d, $c) = @$state;
            if (not exists $group{stringp([@$p, @$d])} or $c < $group{stringp([@$p, @$d])}) {
                $group{stringp([@$p, @$d])} = $c;
            } else {
                next;
            }
            if (not exists $cost_map{stringp($p)} or $c < $cost_map{stringp($p)}) {
                $cost_map{stringp($p)} = $c;
            }

            my @next =
                grep { (not exists $group{stringp([@{$_->[0]}, @{$_->[1]}])}) or $group{stringp([@{$_->[0]}, @{$_->[1]}])} > $_->[2] }
                grep $arr->[$_->[0][0]][$_->[0][1]] ne '#',
                ([ addp($p, $d), $d, $c+1 ], [ $p, rotate90p($d), $c+1000 ], [ $p, rotate270p($d), $c+1000 ]);
            # say Dumper \@next;
            push @next_points, @next;
        }
        # say Dumper \%group;

        # set the cache to our mapped distances
        $map_distance_cache{$arr} = \%cost_map;
        
        return $map_distance_cache{$arr}{stringp($self->{coord})};
    };

sub process_day16_part1 {
    my ($input) = @_;
        # grep defined,
        # flatten_nd
        string_2d_numbers_array
        map_nd selector('map_distance_reindeer'),
    my $map =
        map_nd_indexed graphql_query('{ value map_distance_reindeer(from_value:"S") }'),
        parse_2d_map_array $input;

    my $debug_map =
        string_2d_numbers_array
        map_nd selector('map_distance_reindeer'),
        $map;
    say "debug: map:\n$debug_map";
    writefile(".exp/day16_debug_map", $debug_map);

    my ($shortest_distance) =
        grep defined,
        flatten_nd
        map_nd sub { $_->{value} eq 'E' ? $_->{map_distance_reindeer} : undef },
        $map;
    $shortest_distance;
}
timer sub {
    # say Dumper process_day16_part1($practice_input1);
    # say Dumper process_day16_part1($practice_input2);
    confirm process_day16_part1($practice_input1), 7036;
    confirm process_day16_part1($practice_input2), 11048;
}

debug: map:
      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |
      |  5016|  6017|  6018|  6019|  6020|  6021|  6022|      |  8040|  8039|  8038|  8037|  7036|      |
      |  5015|      |  7017|      |      |      |  7023|      |  9023|      |      |      |  7035|      |
      |  5014|  6015|  6016|  6017|  6018|      |  7024|      |  8022|  8021|  7020|      |  7034|      |
      |  5013|      |      |      |  7019|      |      |      |      |      |  7019|      |  7033|      |
      |  5012|      |  3010|      |  6020|  6019|  6018|  6017|  5016|  6017|  6018|      |  7032|      |
      |  5011|      |  3009|      |      |      |      |      |  5015|      |      |      |  7031|      |
      |  4010|  4009|  3008|  4009|  3010|  4011|  4012|  4013|  4014|  4015|  4016|      |  7030|      |
      |      |      |  3007|      |  3009|      |      |      |      |      |  5017|      |  7029|      |
      |  1004|  2005|  2006|      

1

Warning: Subroutine string_2d_numbers_array redefined at reply input line 1.

Subroutine process_day16_part1 redefined at reply input line 56.


In [93]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/16/input');
    $res = process_day16_part1($input);
    say Dumper $res;
    confirm $res, 103512;
};

debug: map:
      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |


4

In [92]:
say post_answer('2024/day/16/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 16 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [181]:

$graphql::graphql_methods{flood_search_reindeer} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};

        # cache to prevent fetching the same data many times
        state %cache;
        return $cache{$arr}{stringp($self->{coord})} if exists $cache{$arr};

        # prep map data
        my @shape = shape($self->{arr});
        my @start =
            grep defined, flatten_nd
            map_nd selector('where_key.coord'),
            map_nd_indexed graphql_query(qq/{ where_key(key:"v", value:"$params->{from_value}") { coord } }/),
            $arr;

        # say "start: ", Dumper @start;

        my %map;
        my @next_points = @start;
        my $steps = 0;
        while (@next_points) {
            last if $steps++ > 10000;

            @map{map { stringp($_) } @next_points} = (1) x @next_points;
            # say Dumper map { stringp($_) } @next_points;
            
            @next_points =
                uniqp
                grep { not exists $map{stringp $_} }
                map {
                    my $p = $_;
                    
                    grep {
                        my $perp = addp($_, mulp(subp($p, $_), 2));
                        ($arr->[$_->[0]][$_->[1]]{distance} + 1 == $arr->[$p->[0]][$p->[1]]{distance}
                            || $arr->[$_->[0]][$_->[1]]{distance} + 1001 == $arr->[$p->[0]][$p->[1]]{distance})
                        || (defined ($arr->[$perp->[0]][$perp->[1]]{distance})
                            && exists $map{stringp($perp)}
                            && $arr->[$_->[0]][$_->[1]]{distance} + 2 == $arr->[$perp->[0]][$perp->[1]]{distance}) }
                    grep { defined $arr->[$_->[0]][$_->[1]]{distance} }
                    adjacent_points($p, @shape)
                }
                @next_points;
            # say 'next:', Dumper \@next_points;
        }
        # say Dumper $arr;
        # say Dumper \%map;

        # set the cache to our mapped distances
        $cache{$arr} = \%map;
        
        return $cache{$arr}{stringp($self->{coord})};
    };

$graphql::graphql_methods{where_key} = sub {
        my ($self, $params) = @_;
        return (defined ($self->{value}{$params->{key}}) and $self->{value}{$params->{key}} eq $params->{value}) ? $self : undef;
    };

sub process_day16_part2 {
    my ($input) = @_;

    my $map =
        map_nd_indexed graphql_query('{ v:value distance:map_distance_reindeer(from_value:"S") }'),
        parse_2d_map_array $input;

    my $debug_map =
        string_2d_numbers_array
        map_nd selector('distance'),
        $map;
    my $search_map =
        string_2d_map_array
        map_nd sub { (defined and $_ == 1) ? 'O' : ' ' },
        map_nd selector('flood_search_reindeer'),
        map_nd_indexed graphql_query('{ flood_search_reindeer(from_value:"E") }'),
        $map;
    my $overlay_map =
        string_2d_numbers_array
        map_nd_indexed sub { my ($arr, $p) = @_; my ($x,$y) = @$p; $arr->[$x][$y] eq 'O' ? $map->[$x][$y]{distance} : undef },
        parse_2d_map_array
        $search_map;
    say "debug: map:\n$debug_map\ndebug: search_map:\n$search_map\n debug: overlay:\n$overlay_map";
    # say "debug: map2:\n$debug_map";
    writefile(".exp/day16_debug_map", "debug: map:\n$debug_map\ndebug: search_map:\n$search_map\n debug: overlay:\n$overlay_map");

    return scalar (() = $search_map =~ /(O)/g);
}
timer sub {
    # say Dumper process_day16_part2($practice_input1);
    # say Dumper process_day16_part2($practice_input2);
    confirm process_day16_part2($practice_input1), 45;
    confirm process_day16_part2($practice_input2), 64;
}

debug: map:
      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |
      |  5016|  6017|  6018|  6019|  6020|  6021|  6022|      |  8040|  8039|  8038|  8037|  7036|      |
      |  5015|      |  7017|      |      |      |  7023|      |  9023|      |      |      |  7035|      |
      |  5014|  6015|  6016|  6017|  6018|      |  7024|      |  8022|  8021|  7020|      |  7034|      |
      |  5013|      |      |      |  7019|      |      |      |      |      |  7019|      |  7033|      |
      |  5012|      |  3010|      |  6020|  6019|  6018|  6017|  5016|  6017|  6018|      |  7032|      |
      |  5011|      |  3009|      |      |      |      |      |  5015|      |      |      |  7031|      |
      |  4010|  4009|  3008|  4009|  3010|  4011|  4012|  4013|  4014|  4015|  4016|      |  7030|      |
      |      |      |  3007|      |  3009|      |      |      |      |      |  5017|      |  7029|      |
      |  1004|  2005|  2006|      

0

Warning: Subroutine process_day16_part2 redefined at reply input line 62.


In [182]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/16/input');
    $res = process_day16_part2($input);
    say Dumper $res;
    confirm $res, 554;
};

debug: map:
      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |


4

In [184]:
# <595
say post_answer('2024/day/16/answer', 2, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 16 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1